In [1]:
import glob
import numpy as np
import pickle

def print_adjusted_metrics(das_type, treatment, outcome_col = 'class_bin'):
    apparent_res_struct = pickle.load(open(f'./data/results/{treatment}/{outcome_col}/{das_type}_{treatment}_{outcome_col}_apparent_eval.data', "rb"))
    
    boot_files = glob.glob(f'./data/results/{treatment}/{outcome_col}/bootstraps/{das_type}*.data')
    
    n_boot_files = len(boot_files)
    print(f'Processing {n_boot_files} bootstrap files')
    
    brier_score_optimism = np.zeros(n_boot_files)
    auc_optimism = np.zeros(n_boot_files)
    citl_optimism = []
    cal_slope_optimism = []
    
    for idx, boot_file in enumerate(boot_files):
        res_struct = pickle.load(open(boot_file, "rb"))
        
        brier_score_optimism[idx] = res_struct['acc_measure_optimism']
        auc_optimism[idx] = res_struct['auc_optimism']
        citl_optimism.append(res_struct['citl_optimism'])
        cal_slope_optimism.append(res_struct['cal_slope_optimism'])
        
    citl_optimism = np.array(citl_optimism)
    cal_slope_optimism = np.array(cal_slope_optimism)
    
    acc = apparent_res_struct['acc_measure']
    adjusted_brier_score = apparent_res_struct['acc_measure'] - np.mean(brier_score_optimism)
    acc_iqr = apparent_res_struct['acc_measure_iqr']
    acc_lower_bound = adjusted_brier_score - apparent_res_struct['acc_measure_iqr']
    acc_uper_bound = adjusted_brier_score + apparent_res_struct['acc_measure_iqr']
    
    auc = apparent_res_struct['auc']
    adjusted_auc = apparent_res_struct['auc'] - np.mean(auc_optimism)
    auc_iqr = apparent_res_struct['auc_iqr']
    auc_lower_bound = adjusted_auc - apparent_res_struct['auc_iqr']
    auc_upper_bound = adjusted_auc + apparent_res_struct['auc_iqr']
    
    citl = apparent_res_struct['citl']
    citl = np.atleast_1d(citl)
    adjusted_citl = apparent_res_struct['citl'] - np.mean(citl_optimism, axis = 0)
    adjusted_citl = np.atleast_1d(adjusted_citl)
    citl_iqr = apparent_res_struct['citl_iqr']
    citl_iqr = np.atleast_1d(citl_iqr)
    citl_lower_bound = adjusted_citl - apparent_res_struct['citl_iqr']
    citl_lower_bound = np.atleast_1d(citl_lower_bound)
    citl_upper_bound = adjusted_citl + apparent_res_struct['citl_iqr']
    citl_upper_bound = np.atleast_1d(citl_upper_bound)
    
    cal_slope = apparent_res_struct['cal_slope']
    cal_slope = np.atleast_1d(cal_slope)
    adjusted_cal_slope = apparent_res_struct['cal_slope'] - np.mean(cal_slope_optimism, axis = 0)
    adjusted_cal_slope = np.atleast_1d(adjusted_cal_slope)
    cal_slope_iqr = apparent_res_struct['cal_slope_iqr']
    cal_slope_iqr = np.atleast_1d(cal_slope_iqr)
    cal_slope_lower_bound = adjusted_cal_slope - apparent_res_struct['cal_slope_iqr']
    cal_slope_lower_bound = np.atleast_1d(cal_slope_lower_bound)
    cal_slope_upper_bound = adjusted_cal_slope + apparent_res_struct['cal_slope_iqr']
    cal_slope_upper_bound = np.atleast_1d(cal_slope_upper_bound)
        
    print('---------')
    print(f'Performance for {das_type} - {treatment} - {outcome_col}')
    print(f'Acc: {acc:0.3f}')
    print(f'Adjusted Acc Measure: {adjusted_brier_score:0.3f} +/- {acc_iqr:0.3f} (IQR: {acc_lower_bound:0.3f} - {acc_uper_bound:0.3f})')
    print(f'AUC: {auc:0.3f}')
    print(f'Adjusted AUC: {adjusted_auc:0.3f} +/- {auc_iqr:0.3f} (IQR: {auc_lower_bound:0.3f} - {auc_upper_bound:0.3f})')
    
    for idx, _ in enumerate(adjusted_citl):
        print(f'Citl_{idx}: {citl[idx]:0.3f}')
        print(f'Adjusted CITL_{idx}: {adjusted_citl[idx]:0.3f} +/- {citl_iqr[idx]:0.3f} (IQR: {citl_lower_bound[idx]:0.3f} - {citl_upper_bound[idx]:0.3f})')
    for idx, _ in enumerate(adjusted_cal_slope):
        print(f'Slope_{idx}: {cal_slope[idx]:0.3f}')
        print(f'Adjusted Cal Slope_{idx}: {adjusted_cal_slope[idx]:0.3f} +/- {cal_slope_iqr[idx]:0.3f} (IQR: {cal_slope_lower_bound[idx]:0.3f} - {cal_slope_upper_bound[idx]:0.3f})')
    
    print('---------')

In [2]:
def print_adjusted_cont_metrics(das_type, treatment, outcome_col = '2cdas_fu'):
    apparent_res_struct = pickle.load(open(f'./data/results/{treatment}/{outcome_col}/{das_type}_{treatment}_{outcome_col}_apparent_eval.data', "rb"))
    
    boot_files = glob.glob(f'./data/results/{treatment}/{outcome_col}/bootstraps/{das_type}*.data')
    
    n_boot_files = len(boot_files)
    print(f'Processing {n_boot_files} bootstrap files')
    
    mae_optimism = np.zeros(n_boot_files)
    mse_optimism = np.zeros(n_boot_files)
    r2_optimism = np.zeros(n_boot_files)
    
    for idx, boot_file in enumerate(boot_files):
        res_struct = pickle.load(open(boot_file, "rb"))
        
        mae_optimism[idx] = res_struct['boot_apparent_performance']['mae'] - res_struct['mae']
        mse_optimism[idx] = res_struct['boot_apparent_performance']['mse'] - res_struct['mse']
        r2_optimism[idx] = res_struct['boot_apparent_performance']['r2'] - res_struct['r2']
        
    mae = apparent_res_struct['mae']
    adjusted_mae = mae - np.mean(mae_optimism)
    mae_iqr = apparent_res_struct['mae_values_iqr']
    mae_lower_bound = adjusted_mae - mae_iqr
    mae_uper_bound = adjusted_mae + mae_iqr
    
    mse = apparent_res_struct['mse']
    adjusted_mse = mse - np.mean(mse_optimism)
    mse_iqr = apparent_res_struct['mse_values_iqr']
    mse_lower_bound = adjusted_mse - mse_iqr
    mse_uper_bound = adjusted_mse + mse_iqr
    
    r2 = apparent_res_struct['r2']
    adjusted_r2 = r2 - np.mean(r2_optimism)
    r2_iqr = apparent_res_struct['r2_iqr']
    r2_lower_bound = adjusted_r2 - r2_iqr
    r2_uper_bound = adjusted_r2 + r2_iqr
        
    print('---------')
    print(f'Performance for {das_type} - {treatment} - {outcome_col}')
    print(f'MAE: {mae:0.3f}')
    print(f'Adjusted MAE: {adjusted_mae:0.3f} +/- {mae_iqr:0.3f} (IQR: {mae_lower_bound:0.3f} - {mae_uper_bound:0.3f})')
    print(f'MSE: {mse:0.3f}')
    print(f'Adjusted MSE: {adjusted_mse:0.3f} +/- {mse_iqr:0.3f} (IQR: {mse_lower_bound:0.3f} - {mse_uper_bound:0.3f})')
    print(f'R2: {r2:0.3f}')
    print(f'Adjusted R2: {adjusted_r2:0.3f} +/- {r2_iqr:0.3f} (IQR: {r2_lower_bound:0.3f} - {r2_uper_bound:0.3f})')
    print('---------')

In [2]:
print_adjusted_metrics('crp', 'etanercept', outcome_col = 'class_poor')

Processing 500 bootstrap files
---------
Performance for crp - etanercept - class_poor
Acc: 0.140
Adjusted Acc Measure: 0.148 +/- 0.000 (IQR: 0.147 - 0.148)
AUC: 0.703
Adjusted AUC: 0.658 +/- 0.002 (IQR: 0.656 - 0.660)
Citl_0: -0.001
Adjusted CITL_0: -0.315 +/- 0.017 (IQR: -0.332 - -0.299)
Slope_0: 1.000
Adjusted Cal Slope_0: 0.765 +/- 0.011 (IQR: 0.755 - 0.776)
---------


In [3]:
print_adjusted_metrics('esr', 'etanercept', outcome_col = 'class_poor')

Processing 500 bootstrap files
---------
Performance for esr - etanercept - class_poor
Acc: 0.147
Adjusted Acc Measure: 0.157 +/- 0.000 (IQR: 0.157 - 0.158)
AUC: 0.717
Adjusted AUC: 0.671 +/- 0.002 (IQR: 0.669 - 0.673)
Citl_0: 0.003
Adjusted CITL_0: -0.277 +/- 0.014 (IQR: -0.291 - -0.263)
Slope_0: 1.001
Adjusted Cal Slope_0: 0.761 +/- 0.011 (IQR: 0.751 - 0.772)
---------


In [4]:
print_adjusted_metrics('crp', 'adalimumab', outcome_col = 'class_poor')

Processing 500 bootstrap files
---------
Performance for crp - adalimumab - class_poor
Acc: 0.127
Adjusted Acc Measure: 0.141 +/- 0.000 (IQR: 0.141 - 0.141)
AUC: 0.746
Adjusted AUC: 0.683 +/- 0.001 (IQR: 0.682 - 0.684)
Citl_0: 0.000
Adjusted CITL_0: -0.402 +/- 0.005 (IQR: -0.407 - -0.397)
Slope_0: 1.001
Adjusted Cal Slope_0: 0.696 +/- 0.004 (IQR: 0.692 - 0.700)
---------


In [5]:
print_adjusted_metrics('esr', 'adalimumab', outcome_col = 'class_poor')

Processing 500 bootstrap files
---------
Performance for esr - adalimumab - class_poor
Acc: 0.134
Adjusted Acc Measure: 0.152 +/- 0.001 (IQR: 0.152 - 0.153)
AUC: 0.780
Adjusted AUC: 0.722 +/- 0.002 (IQR: 0.720 - 0.725)
Citl_0: -0.001
Adjusted CITL_0: -0.308 +/- 0.011 (IQR: -0.319 - -0.297)
Slope_0: 0.996
Adjusted Cal Slope_0: 0.714 +/- 0.015 (IQR: 0.699 - 0.729)
---------


In [6]:
print_adjusted_metrics('crp', 'eta_ada', outcome_col = 'class_poor')

Processing 500 bootstrap files
---------
Performance for crp - eta_ada - class_poor
Acc: 0.141
Adjusted Acc Measure: 0.146 +/- 0.000 (IQR: 0.146 - 0.146)
AUC: 0.680
Adjusted AUC: 0.648 +/- 0.001 (IQR: 0.647 - 0.649)
Citl_0: -0.001
Adjusted CITL_0: -0.247 +/- 0.009 (IQR: -0.256 - -0.237)
Slope_0: 0.999
Adjusted Cal Slope_0: 0.819 +/- 0.006 (IQR: 0.813 - 0.825)
---------


In [7]:
print_adjusted_metrics('esr', 'eta_ada', outcome_col = 'class_poor')

Processing 500 bootstrap files
---------
Performance for esr - eta_ada - class_poor
Acc: 0.148
Adjusted Acc Measure: 0.154 +/- 0.000 (IQR: 0.154 - 0.154)
AUC: 0.708
Adjusted AUC: 0.677 +/- 0.001 (IQR: 0.675 - 0.678)
Citl_0: -0.000
Adjusted CITL_0: -0.198 +/- 0.012 (IQR: -0.210 - -0.186)
Slope_0: 0.999
Adjusted Cal Slope_0: 0.834 +/- 0.007 (IQR: 0.826 - 0.841)
---------


In [6]:
print_adjusted_metrics('crp', 'etanercept', outcome_col = 'class')

Processing 500 bootstrap files
---------
Performance for crp - etanercept - class
Acc: 0.542
Adjusted Acc Measure: 0.511 +/- 0.001 (IQR: 0.510 - 0.513)
AUC: 0.693
Adjusted AUC: 0.654 +/- 0.001 (IQR: 0.653 - 0.655)
Citl_0: -0.442
Adjusted CITL_0: -0.420 +/- 0.002 (IQR: -0.423 - -0.418)
Citl_1: -1.042
Adjusted CITL_1: -1.102 +/- 0.009 (IQR: -1.110 - -1.093)
Slope_0: 0.694
Adjusted Cal Slope_0: 0.573 +/- 0.003 (IQR: 0.569 - 0.576)
Slope_1: 0.676
Adjusted Cal Slope_1: 0.531 +/- 0.009 (IQR: 0.522 - 0.541)
---------


In [7]:
print_adjusted_metrics('esr', 'etanercept', outcome_col = 'class')

Processing 500 bootstrap files
---------
Performance for esr - etanercept - class
Acc: 0.547
Adjusted Acc Measure: 0.513 +/- 0.006 (IQR: 0.508 - 0.519)
AUC: 0.704
Adjusted AUC: 0.665 +/- 0.001 (IQR: 0.665 - 0.666)
Citl_0: -0.464
Adjusted CITL_0: -0.407 +/- 0.010 (IQR: -0.416 - -0.397)
Citl_1: -1.113
Adjusted CITL_1: -1.126 +/- 0.005 (IQR: -1.131 - -1.121)
Slope_0: 0.740
Adjusted Cal Slope_0: 0.609 +/- 0.005 (IQR: 0.604 - 0.614)
Slope_1: 0.700
Adjusted Cal Slope_1: 0.551 +/- 0.008 (IQR: 0.543 - 0.558)
---------


In [8]:
print_adjusted_metrics('crp', 'adalimumab', outcome_col = 'class')

Processing 500 bootstrap files
---------
Performance for crp - adalimumab - class
Acc: 0.572
Adjusted Acc Measure: 0.519 +/- 0.009 (IQR: 0.510 - 0.527)
AUC: 0.714
Adjusted AUC: 0.656 +/- 0.001 (IQR: 0.655 - 0.657)
Citl_0: -0.443
Adjusted CITL_0: -0.415 +/- 0.001 (IQR: -0.417 - -0.414)
Citl_1: -1.160
Adjusted CITL_1: -1.209 +/- 0.004 (IQR: -1.213 - -1.204)
Slope_0: 0.434
Adjusted Cal Slope_0: 0.284 +/- 0.003 (IQR: 0.281 - 0.287)
Slope_1: 0.569
Adjusted Cal Slope_1: 0.443 +/- 0.003 (IQR: 0.439 - 0.446)
---------


In [9]:
print_adjusted_metrics('esr', 'adalimumab', outcome_col = 'class')

Processing 500 bootstrap files
---------
Performance for esr - adalimumab - class
Acc: 0.585
Adjusted Acc Measure: 0.524 +/- 0.011 (IQR: 0.513 - 0.535)
AUC: 0.745
Adjusted AUC: 0.684 +/- 0.002 (IQR: 0.682 - 0.686)
Citl_0: -0.393
Adjusted CITL_0: -0.296 +/- 0.003 (IQR: -0.299 - -0.292)
Citl_1: -1.181
Adjusted CITL_1: -1.160 +/- 0.018 (IQR: -1.178 - -1.142)
Slope_0: 0.532
Adjusted Cal Slope_0: 0.354 +/- 0.008 (IQR: 0.346 - 0.361)
Slope_1: 0.687
Adjusted Cal Slope_1: 0.531 +/- 0.007 (IQR: 0.524 - 0.537)
---------


In [12]:
print_adjusted_metrics('esr', 'eta_ada', outcome_col = 'class')

Processing 500 bootstrap files
---------
Performance for esr - eta_ada - class
Acc: 0.546
Adjusted Acc Measure: 0.522 +/- 0.002 (IQR: 0.520 - 0.524)
AUC: 0.694
Adjusted AUC: 0.665 +/- 0.001 (IQR: 0.664 - 0.666)
Citl_0: -0.444
Adjusted CITL_0: -0.394 +/- 0.005 (IQR: -0.398 - -0.389)
Citl_1: -1.088
Adjusted CITL_1: -1.101 +/- 0.009 (IQR: -1.110 - -1.092)
Slope_0: 0.699
Adjusted Cal Slope_0: 0.593 +/- 0.004 (IQR: 0.589 - 0.598)
Slope_1: 0.740
Adjusted Cal Slope_1: 0.635 +/- 0.006 (IQR: 0.629 - 0.641)
---------
